In [1]:
import urllib2
import json
import datetime
import csv
import time

In [2]:
app_id = "1568109056740159"
app_secret = "f68cb213baed613afd40f5d2b176d685"

access_token = app_id + "|" + app_secret

In [3]:
page_id = 'nytimes'

In [4]:
def testFacebookPageData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    req = urllib2.Request(url)
    response = urllib2.urlopen(req)
    data = json.loads(response.read())
    
    print json.dumps(data, indent=4, sort_keys=True)
    

testFacebookPageData(page_id, access_token)

{
    "id": "5281959998", 
    "name": "The New York Times"
}


In [5]:
def request_until_succeed(url):
    req = urllib2.Request(url)
    success = False
    while success is False:
        try: 
            response = urllib2.urlopen(req)
            if response.getcode() == 200:
                success = True
        except Exception, e:
            print e
            time.sleep(5)
            
            print "Error for URL %s: %s" % (url, datetime.datetime.now())

    return response.read()

In [6]:
def testFacebookPageFeedData(page_id, access_token):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.4"
    node = "/" + page_id + "/feed" # changed
    parameters = "/?access_token=%s" % access_token
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    print json.dumps(data, indent=4, sort_keys=True)
    

testFacebookPageFeedData(page_id, access_token)

{
    "data": [
        {
            "created_time": "2016-09-26T15:55:00+0000", 
            "id": "5281959998_10150911766579999", 
            "message": "A week later, a North Carolina law would have stopped the release of police videos of a Charlotte shooting."
        }, 
        {
            "created_time": "2016-09-26T15:48:26+0000", 
            "id": "5281959998_10150911774719999", 
            "message": "Join Nicholas Kristof live as he takes questions about tonight's presidential debate.", 
            "story": "The New York Times shared Nicholas Kristof's live video."
        }, 
        {
            "created_time": "2016-09-26T15:25:00+0000", 
            "id": "5281959998_10150911663999999", 
            "message": "Hillary Clinton shouldn't waste her time in the debate explaining to people what a scoundrel Donald J. Trump is, Nicolle Wallace writes in The New York Times Opinion Section ."
        }, 
        {
            "created_time": "2016-09-26T15:10:14+0000", 


In [8]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com"
    node = "/" + page_id + "/feed" 
    parameters = "/?fields=message,link,created_time,type,name,id,likes.limit(1).summary(true),comments.limit(1).summary(true),shares&limit=%s&access_token=%s" % (num_statuses, access_token) # changed
    url = base + node + parameters
    
    # retrieve data
    data = json.loads(request_until_succeed(url))
    
    return data

test_status = getFacebookPageFeedData(page_id, access_token, 1)["data"][0]
print json.dumps(test_status, indent=4, sort_keys=True)

{
    "comments": {
        "data": [
            {
                "can_remove": false, 
                "created_time": "2016-09-26T16:01:03+0000", 
                "from": {
                    "id": "10207511727337072", 
                    "name": "Maureen Sheridan"
                }, 
                "id": "10150911766579999_10150911784019999", 
                "like_count": 0, 
                "message": "Wow. This pretty much ensures I'll Never travel to North Carolina. Trending backwards as fast as it can. Regressive police state, anyone?", 
                "user_likes": false
            }
        ], 
        "paging": {
            "cursors": {
                "after": "MTYZD", 
                "before": "MTYZD"
            }, 
            "next": "https://graph.facebook.com/v2.2/5281959998_10150911766579999/comments?access_token=1568109056740159%7Cf68cb213baed613afd40f5d2b176d685&summary=true&limit=1&after=MTYZD"
        }, 
        "summary": {
            "order": "ranked

In [9]:

def processFacebookPageFeedStatus(status):
    
    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.
    
    # Additionally, some items may not always exist,
    # so must check for existence first
    
    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else status['message'].encode('utf-8')
    link_name = '' if 'name' not in status.keys() else status['name'].encode('utf-8')
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else status['link']
    
    
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.
    
    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    # Nested items require chaining dictionary keys.
    
    num_likes = 0 if 'likes' not in status.keys() else status['likes']['summary']['total_count']
    num_comments = 0 if 'comments' not in status.keys() else status['comments']['summary']['total_count']
    num_shares = 0 if 'shares' not in status.keys() else status['shares']['count']
    
    # return a tuple of all processed data
    return (status_id, status_message, link_name, status_type, status_link,
           status_published, num_likes, num_comments, num_shares)

processed_test_status = processFacebookPageFeedStatus(test_status)
print processed_test_status

(u'5281959998_10150911766579999', 'A week later, a North Carolina law would have stopped the release of police videos of a Charlotte shooting.', 'Video of Charlotte Police Shooting Could Be the Last Released in North Carolina', u'link', u'http://nyti.ms/2dtOdqh', '2016-09-26 10:55:00', 17, 16, 8)


In [10]:
def scrapeFacebookPageFeedStatus(page_id, access_token):
    with open('%s_facebook_statuses.csv' % page_id, 'wb') as file:
        w = csv.writer(file)
        w.writerow(["status_id", "status_message", "link_name", "status_type", "status_link",
           "status_published", "num_likes", "num_comments", "num_shares"])
        
        has_next_page = True
        num_processed = 0   # keep a count on how many we've processed
        scrape_starttime = datetime.datetime.now()
        
        print "Scraping %s Facebook Page: %s\n" % (page_id, scrape_starttime)
        
        statuses = getFacebookPageFeedData(page_id, access_token, 100)
        
        while has_next_page:
            for status in statuses['data']:
                w.writerow(processFacebookPageFeedStatus(status))
                
                # output progress occasionally to make sure code is not stalling
                num_processed += 1
                if num_processed % 1000 == 0:
                    print "%s Statuses Processed: %s" % (num_processed, datetime.datetime.now())
                    
            # if there is no next page, we're done.
            if 'paging' in statuses.keys():
                statuses = json.loads(request_until_succeed(statuses['paging']['next']))
            else:
                has_next_page = False
                
        
        print "\nDone!\n%s Statuses Processed in %s" % (num_processed, datetime.datetime.now() - scrape_starttime)


scrapeFacebookPageFeedStatus(page_id, access_token)

Scraping nytimes Facebook Page: 2016-09-26 23:06:54.995409

1000 Statuses Processed: 2016-09-26 23:07:31.989418
2000 Statuses Processed: 2016-09-26 23:08:23.947880
3000 Statuses Processed: 2016-09-26 23:09:09.455466
4000 Statuses Processed: 2016-09-26 23:09:59.694442
5000 Statuses Processed: 2016-09-26 23:10:47.077318
6000 Statuses Processed: 2016-09-26 23:11:36.989625
7000 Statuses Processed: 2016-09-26 23:12:37.583012


KeyError: 'total_count'

In [14]:
with open('nytimes_facebook_statuses.csv') as fl:
    print fl.readline()

status_id,status_message,link_name,status_type,status_link,status_published,num_likes,num_comments,num_shares



In [12]:
fl

<open file 'nytimes_facebook_statuses.csv', mode 'r' at 0x10cd8bd20>